In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import tqdm
import decoupler as dc
import numpy as np

In [2]:
protein_network_df = pd.read_csv('../../protein_network_analysis/9606.protein.physical.links.v12.0.txt.gz', sep=' ')

In [3]:
protein_info = pd.read_csv('../../protein_network_analysis/9606.protein.info.v12.0.txt.gz', sep='\t')
protein_info_dict = dict(zip(protein_info['#string_protein_id'], protein_info['preferred_name']))

In [4]:
protein_network_df['protein1_name'] = protein_network_df.protein1.map(protein_info_dict)

In [5]:
protein_network_df['protein2_name'] = protein_network_df.protein2.map(protein_info_dict)

In [6]:
cluster_index = ['TRNAU1AP', 'RBM5', 'HNRNPF', 'MBNL2', 'LSM10', 'PRPF4', 'RBM22',
       'RBM42', 'RNPS1', 'LSM1', 'PCBP1', 'RBM10', 'SMNDC1', 'CNOT2', 'PCBP3',
       'YWHAG', 'YTHDF1', 'MBNL1', 'EIF4B', 'RBM11', 'AHNAK', 'ZMAT3', 'STAU2',
       'WDR6', 'DZIP3', 'TPD52L2', 'GRB2', 'PPIA', 'ZC3HAV1', 'RPS19BP1',
       'SRP68', 'LGALS3', 'RBM4', 'HNRNPH2', 'PARN', 'RBM14', 'PPP1CA',
       'SCAF8', 'NUDT21', 'CPSF6']

In [7]:
interest=cluster_index.copy()

In [8]:
interactors_dict = protein_network_df[protein_network_df['protein1_name'].isin(interest)].groupby('protein1_name')['protein2_name'].apply(list).to_dict()

In [9]:
msigdb = dc.get_resource('MSigDB')
msigdb

/tscc/nfs/home/pjaganna1/anaconda3/envs/decoupler/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,genesymbol,collection,geneset
0,MAFF,chemical_and_genetic_perturbations,BOYAULT_LIVER_CANCER_SUBCLASS_G56_DN
1,MAFF,chemical_and_genetic_perturbations,ELVIDGE_HYPOXIA_UP
2,MAFF,chemical_and_genetic_perturbations,NUYTTEN_NIPP1_TARGETS_DN
3,MAFF,immunesigdb,GSE17721_POLYIC_VS_GARDIQUIMOD_4H_BMDC_DN
4,MAFF,chemical_and_genetic_perturbations,SCHAEFFER_PROSTATE_DEVELOPMENT_12HR_UP
...,...,...,...
3838543,PRAMEF22,go_biological_process,GOBP_POSITIVE_REGULATION_OF_CELL_POPULATION_PR...
3838544,PRAMEF22,go_biological_process,GOBP_APOPTOTIC_PROCESS
3838545,PRAMEF22,go_biological_process,GOBP_REGULATION_OF_CELL_DEATH
3838546,PRAMEF22,go_biological_process,GOBP_NEGATIVE_REGULATION_OF_DEVELOPMENTAL_PROCESS


In [10]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection']=='go_biological_process']

# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]

msigdb

,genesymbol,collection,geneset
33,MAFF,go_biological_process,GOBP_EMBRYO_DEVELOPMENT
44,MAFF,go_biological_process,GOBP_POSITIVE_REGULATION_OF_RNA_METABOLIC_PROCESS
82,MAFF,go_biological_process,GOBP_REGULATION_OF_EPITHELIAL_CELL_DIFFERENTIA...
94,MAFF,go_biological_process,GOBP_EMBRYO_DEVELOPMENT_ENDING_IN_BIRTH_OR_EGG...
108,MAFF,go_biological_process,GOBP_IN_UTERO_EMBRYONIC_DEVELOPMENT
...,...,...,...
3838543,PRAMEF22,go_biological_process,GOBP_POSITIVE_REGULATION_OF_CELL_POPULATION_PR...
3838544,PRAMEF22,go_biological_process,GOBP_APOPTOTIC_PROCESS
3838545,PRAMEF22,go_biological_process,GOBP_REGULATION_OF_CELL_DEATH
3838546,PRAMEF22,go_biological_process,GOBP_NEGATIVE_REGULATION_OF_DEVELOPMENTAL_PROCESS


In [11]:
msigdb.loc[:, 'geneset'] = [name.split('GOBP_')[1] for name in msigdb['geneset']]
msigdb

,genesymbol,collection,geneset
33,MAFF,go_biological_process,EMBRYO_DEVELOPMENT
44,MAFF,go_biological_process,POSITIVE_REGULATION_OF_RNA_METABOLIC_PROCESS
82,MAFF,go_biological_process,REGULATION_OF_EPITHELIAL_CELL_DIFFERENTIATION
94,MAFF,go_biological_process,EMBRYO_DEVELOPMENT_ENDING_IN_BIRTH_OR_EGG_HATC...
108,MAFF,go_biological_process,IN_UTERO_EMBRYONIC_DEVELOPMENT
...,...,...,...
3838543,PRAMEF22,go_biological_process,POSITIVE_REGULATION_OF_CELL_POPULATION_PROLIFE...
3838544,PRAMEF22,go_biological_process,APOPTOTIC_PROCESS
3838545,PRAMEF22,go_biological_process,REGULATION_OF_CELL_DEATH
3838546,PRAMEF22,go_biological_process,NEGATIVE_REGULATION_OF_DEVELOPMENTAL_PROCESS


In [12]:
r_list = []
num_top_terms = []
go_pvals = pd.DataFrame()
for rbp in tqdm.tqdm(cluster_index):
    r_list.append(rbp)

    
    top_genes = pd.DataFrame(interactors_dict[rbp]).copy()
    top_genes = top_genes.rename(columns={0: 'gene_name'})
    top_genes = top_genes.set_index('gene_name')
    top_genes['group'] = 'interactor'
    
    # Run ora
    enr_pvals = dc.get_ora_df(
        df=top_genes,
        net=msigdb,
        source='geneset',
        target='genesymbol'
    )
    
    total_num_terms = enr_pvals[enr_pvals['FDR p-value']<0.05].shape[0]
    top_pct = int(0.01*total_num_terms)
    enr_pvals['-log10 FDR p-value'] = -np.log10(enr_pvals['FDR p-value'])
    print(rbp, top_genes.shape[0], enr_pvals[enr_pvals['FDR p-value']<0.05].shape[0])
    keep_term = enr_pvals[enr_pvals['FDR p-value']<0.05].sort_values(by=['-log10 FDR p-value', 'Combined score'],ascending=[False, False]).head(10)
    num_top_terms.append(enr_pvals[enr_pvals['FDR p-value']<0.05].shape[0])
    keep_term['RBP'] = rbp
    go_pvals = pd.concat([keep_term
                          ,go_pvals])

  2%|▎         | 1/40 [00:00<00:27,  1.42it/s]

TRNAU1AP 80 78


  5%|▌         | 2/40 [00:01<00:26,  1.43it/s]

RBM5 107 121


  8%|▊         | 3/40 [00:02<00:26,  1.41it/s]

...

SCAF8 75 111


 98%|█████████▊| 39/40 [00:27<00:00,  1.39it/s]

NUDT21 202 365


100%|██████████| 40/40 [00:28<00:00,  1.40it/s]

CPSF6 391 362


In [13]:
go_pvals.head()

,Term,Set size,Overlap ratio,p-value,FDR p-value,Odds ratio,Combined score,Features,-log10 FDR p-value,RBP
943,MRNA_PROCESSING,465,0.395699,1.538130e-204,4.535944e-201,37.314972,17511.796455,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BU...,200.343332,CPSF6
940,MRNA_METABOLIC_PROCESS,706,0.277620,7.203118e-184,1.062100e-180,27.449672,11575.557956,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;ATXN2L;B...,179.973835,CPSF6
2683,RNA_PROCESSING,935,0.213904,3.428595e-163,3.370309e-160,21.337818,7982.247678,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BU...,159.472330,CPSF6
2685,RNA_SPLICING,432,0.347222,6.002141e-153,4.425079e-150,28.196213,9882.868765,ACIN1;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BUD13;C...,149.354079,CPSF6
2687,RNA_SPLICING_VIA_TRANSESTERIFICATION_REACTIONS,295,0.427119,1.299003e-140,7.661517e-138,31.772835,10234.040356,ALYREF;AQR;BCAS2;BUD13;CDC40;CDC5L;CELF1;CRNKL...,137.115685,CPSF6


In [14]:
go_pvals_copy = go_pvals.copy()

In [15]:
import pickle

In [18]:
### Dictionary with broader categorization of functional terms
with open('functional_groups_dictionary_interactor_analysis.pkl', 'rb') as file:
    functional_groups = pickle.load(file)

In [19]:
functional_groups.keys()

dict_keys(['Cytoskeleton and Cell Structure', 'Cell Cycle and Division', 'Gene Expression and RNA Processing', 'Ribosome and Translation Biogenesis', 'Metabolism and Biosynthesis', 'Signal Transduction and Regulation', 'Immune and Inflammatory Response', 'Cellular Response to Stimuli', 'Protein Modification and Degradation', 'Apoptosis and Cell Death', 'Transport and Localization', 'Development and Morphogenesis'])

In [20]:
def assign_larger_function(df):
    for key in functional_groups:
        if df.Term in functional_groups[key]:
            return key
            

In [21]:
go_pvals_copy['Functional_group'] = go_pvals_copy.apply(assign_larger_function, axis=1)

In [22]:
go_pvals_copy = go_pvals_copy.set_index('RBP')

In [23]:
go_pvals_copy.head()

,Term,Set size,Overlap ratio,p-value,FDR p-value,Odds ratio,Combined score,Features,-log10 FDR p-value,Functional_group
RBP,,,,,,,,,,
CPSF6,MRNA_PROCESSING,465,0.395699,1.538130e-204,4.535944e-201,37.314972,17511.796455,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BU...,200.343332,Gene Expression and RNA Processing
CPSF6,MRNA_METABOLIC_PROCESS,706,0.277620,7.203118e-184,1.062100e-180,27.449672,11575.557956,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;ATXN2L;B...,179.973835,Gene Expression and RNA Processing
CPSF6,RNA_PROCESSING,935,0.213904,3.428595e-163,3.370309e-160,21.337818,7982.247678,ACIN1;ADAR;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BU...,159.472330,Gene Expression and RNA Processing
CPSF6,RNA_SPLICING,432,0.347222,6.002141e-153,4.425079e-150,28.196213,9882.868765,ACIN1;AKAP17A;ALYREF;AQR;ARL6IP4;BCAS2;BUD13;C...,149.354079,Gene Expression and RNA Processing
CPSF6,RNA_SPLICING_VIA_TRANSESTERIFICATION_REACTIONS,295,0.427119,1.299003e-140,7.661517e-138,31.772835,10234.040356,ALYREF;AQR;BCAS2;BUD13;CDC40;CDC5L;CELF1;CRNKL...,137.115685,Gene Expression and RNA Processing


In [26]:
### labeled according to the clustering analysis
cluster_index_dict = {'TRNAU1AP': 'cluster1', 'RBM5': 'cluster1', 'HNRNPF': 'cluster1', 'MBNL2': 'cluster1', 'LSM10': 'cluster2', 'PRPF4': 'cluster2', 'RBM22': 'cluster2',
       'RBM42': 'cluster2', 'RNPS1': 'cluster2', 'LSM1': 'cluster2', 'PCBP1': 'cluster2', 'RBM10': 'cluster2', 'SMNDC1': 'cluster2', 'CNOT2': 'cluster3', 'PCBP3': 'cluster3',
       'YWHAG': 'cluster3', 'YTHDF1': 'cluster3', 'MBNL1': 'cluster3', 'EIF4B': 'cluster3', 'RBM11': 'cluster3', 'AHNAK': 'cluster3', 'ZMAT3': 'cluster3', 'STAU2': 'cluster3',
       'WDR6': 'cluster3', 'DZIP3': 'cluster3', 'TPD52L2': 'cluster3', 'GRB2': 'cluster3', 'PPIA': 'cluster4', 'ZC3HAV1': 'cluster4', 'RPS19BP1': 'cluster4',
       'SRP68': 'cluster4', 'LGALS3': 'cluster4', 'RBM4': 'cluster4', 'HNRNPH2': 'cluster4', 'PARN': 'cluster5', 'RBM14': 'cluster5', 'PPP1CA': 'cluster5',
       'SCAF8': 'cluster5', 'NUDT21': 'cluster6', 'CPSF6':'cluster6'}

Fraction calculation was wrong, fixed in in this version

In [ ]:
to_plot = go_pvals_copy.groupby(['RBP', 'Functional_group']).size().reset_index(name='Term_count')
to_plot['RBP'] = pd.Categorical(to_plot['RBP'], categories=cluster_index, ordered=True)
to_plot['cluster_id'] = to_plot['RBP'].map(cluster_index_dict)
total_terms_per_category = to_plot.groupby('RBP')['Term_count'].transform('sum')
# Step 2: Calculate the fraction of terms within each functional category
to_plot['Term_fraction'] = to_plot['Term_count'] / total_terms_per_category
fig, axes = plt.subplots(1, 2, sharey=True,gridspec_kw=dict(width_ratios=[5, 1]), figsize=(3,8))

# Plot using the term count as the size
sns.set_style('white')

# Use scatter plot with Term_count as size
function_scatter = sns.scatterplot(
    data=to_plot,
    y=to_plot.RBP,
    x=to_plot.Functional_group,
    size=to_plot.Term_fraction,
    sizes=(6, 60),  # Adjust min and max circle sizes as needed,
    ax=axes[0],
    hue=to_plot.cluster_id
)
sns.barplot(y=r_list, x=num_top_terms, ax=axes[1], color='black')
# Move legend outside
# function_scatter.legend(
#     bbox_to_anchor=(-0.4, 1),  
#     loc='upper right',  # Anchor legend to the upper left corner
#     title="Term Count Legend"
# )

#axes[1].grid()
axes[0].set_ylabel('total_enriched_terms')
plt.setp(function_scatter.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=8)
plt.legend(bbox_to_anchor=(1.85, 1), loc='upper left', title="Term Count Legend")
plt.savefig('./figures/Jaccard_cluster_allpasseq_decoupler_noproteinlen.png')
plt.savefig('./figures/Jaccard_cluster_allpasseq_decoupler_noproteinlen.svg')
plt.savefig('./figures/Jaccard_cluster_allpasseq_decoupler_noproteinlen.jpg')
# plt.show()


In [ ]:
function_scatter.legend(
    bbox_to_anchor=(-0.4, 1),  
    loc='upper right',  # Anchor legend to the upper left corner
    title="Term Count Legend"
)